In [1]:
import pandas as pd
import  os
import numpy as np
import psycopg2
import glob
import openpyxl

In [3]:
df = pd.read_excel("Bill.xlsx")
df.head()

,Bill Date,Due Date,Bill ID,Vendor Name,Payment Terms,Payment Terms Label,Bill Number,PurchaseOrder,Currency Code,Exchange Rate,...,Item Total,Is Billable,Rate,Purchase Order Number,Tax ID,Tax Name,Tax Percentage,Tax Type,Customer Name,CF.Customer Invoice #
0,2021-02-25,2021-02-25,2620362000001249811,First Manpower - GHS,0,Due on Receipt,GHAFM-202102-02018,NaN,GHS,1.0,...,15975.72,False,15975.72,NaN,NaN,NaN,NaN,NaN,NaN,GHAFM-202102-02018
1,2021-07-01,2021-07-01,2620362000001249827,MSC - GHS,0,NaN,PILIN333072021MSG,INV-000495,GHS,1.0,...,3637.10,False,3637.10,NaN,NaN,NaN,NaN,NaN,NaN,INV-000495
2,2021-07-01,2021-07-01,2620362000001249843,Michael Quartey Transport Service - GHS,0,NaN,MQTS0630,INV-000496,GHS,1.0,...,1600.00,False,1600.00,NaN,NaN,NaN,NaN,NaN,NaN,INV-000496
3,2021-07-02,2021-07-02,2620362000001249859,Ghana Revenue Authority - GHS,0,NaN,TMA1-L-2021-0219665,INV-000496,GHS,1.0,...,10.00,False,10.00,NaN,NaN,NaN,NaN,NaN,NaN,INV-000496
4,2021-06-26,2021-06-26,2620362000001249875,Michael Quartey Transport Service - GHS,0,NaN,MQTS0626,INV-JSA000002,GHS,1.0,...,9200.00,False,9200.00,NaN,NaN,NaN,NaN,NaN,NaN,INV-JSA000002


In [5]:
df.columns = [x.lower().replace(" ","_").replace("%","").replace("&","").replace(".","").replace("#","") for x in df.columns]
df.columns

Index(['bill_date', 'due_date', 'bill_id', 'vendor_name', 'payment_terms',
       'payment_terms_label', 'bill_number', 'purchaseorder', 'currency_code',
       'exchange_rate', 'subtotal', 'total', 'balance', 'vendor_notes',
       'adjustment', 'adjustment_description', 'branch_id', 'branch_name',
       'is_inclusive_tax', 'bill_status', 'item_name', 'account',
       'account_code', 'description', 'quantity', 'tax_amount', 'item_total',
       'is_billable', 'rate', 'purchase_order_number', 'tax_id', 'tax_name',
       'tax_percentage', 'tax_type', 'customer_name', 'cfcustomer_invoice_'],
      dtype='object')

In [6]:
df.dtypes

bill_date                  object
due_date                   object
bill_id                     int64
vendor_name                object
payment_terms               int64
payment_terms_label        object
bill_number                object
purchaseorder              object
currency_code              object
exchange_rate             float64
subtotal                  float64
total                     float64
balance                   float64
vendor_notes               object
adjustment                  int64
adjustment_description     object
branch_id                   int64
branch_name                object
is_inclusive_tax             bool
bill_status                object
item_name                  object
account                    object
account_code                int64
description                object
quantity                    int64
tax_amount                float64
item_total                float64
is_billable                  bool
rate                      float64
purchase_order

In [7]:
replacements = {
    'object' : 'varchar',
    'float64' : 'float',
    'int64' : 'float',
    'datetime64[ns]': 'varchar',
    'datetime64': 'timestamp',
    'bool' : 'varchar'
}
replacements

{'object': 'varchar',
 'float64': 'float',
 'int64': 'float',
 'datetime64[ns]': 'varchar',
 'datetime64': 'timestamp',
 'bool': 'varchar'}

In [8]:
col_str = ", ".join("{} {}".format(n, d) for (n, d) in zip(df.columns, df.dtypes.replace(replacements)))
col_str

'bill_date varchar, due_date varchar, bill_id float, vendor_name varchar, payment_terms float, payment_terms_label varchar, bill_number varchar, purchaseorder varchar, currency_code varchar, exchange_rate float, subtotal float, total float, balance float, vendor_notes varchar, adjustment float, adjustment_description varchar, branch_id float, branch_name varchar, is_inclusive_tax varchar, bill_status varchar, item_name varchar, account varchar, account_code float, description varchar, quantity float, tax_amount float, item_total float, is_billable varchar, rate float, purchase_order_number varchar, tax_id float, tax_name varchar, tax_percentage float, tax_type varchar, customer_name float, cfcustomer_invoice_ varchar'

In [16]:
conn_string = "host= db.app.production.jetstreamafrica.com \
                dbname='jetstream-finance' \
                    user='ujetstreamfinance' password='Smehr5A7FGw4jEX@' "

conn = psycopg2.connect(conn_string)
cursor = conn.cursor()

print('opened database successfully')

opened database successfully


In [17]:
cursor.execute("drop table if exists Bills;")

In [18]:
cursor.execute("create table Bills \
    (bill_date varchar, due_date varchar, bill_id float, vendor_name varchar, payment_terms float, payment_terms_label varchar, bill_number varchar, purchaseorder varchar, currency_code varchar, exchange_rate float, subtotal float, total float, balance float, vendor_notes varchar, adjustment float, adjustment_description varchar, branch_id float, branch_name varchar, is_inclusive_tax varchar, bill_status varchar, item_name varchar, account varchar, account_code float, description varchar, quantity float, tax_amount float, item_total float, is_billable varchar, rate float, purchase_order_number varchar, tax_id float, tax_name varchar, tax_percentage float, tax_type varchar, customer_name float, cfcustomer_invoice_ varchar)")

In [19]:
df.to_csv('Bills', header=df.columns, index=False, encoding='utf-8')
my_file = open("Bills")
print('file opened in memory')

file opened in memory


In [20]:
SQL_STATEMENT = """
COPY Bills FROM STDIN WITH
   CSV
   HEADER
   DELIMITER AS ','
"""
cursor.copy_expert(sql = SQL_STATEMENT, file=my_file)
print('file copied to db')

file copied to db


In [21]:
cursor.execute("grant select on table Payment_made to public")
conn.commit()

cursor.close()
print('table payment made imported successfully')

table payment made imported successfully
